In [36]:
# import nltk
# import streamlit as st
# import pandas as pd
# import spacy
# import pyinflect
# import random     
# import numpy as np



#from create_exercise import Create_exercise

# nlp = spacy.load("en_core_web_sm") 

In [37]:
import nltk
import streamlit as st
import pandas as pd
import spacy
import pyinflect
import random     
import numpy as np

from tqdm import tqdm

nlp = spacy.load("en_core_web_sm") 

# Главный класс приложения
class Create_exercise:
    def __init__(self, task='', answer=''):
        #self.df = df
        #self.options=options
        self.task = task
        self.answer = answer
        self.exercise_type = exercise_type

 # Загрузка текста   
    def get_text(self):
        with open('Little_Red_Cap_ Jacob_and_Wilhelm_Grimm.txt') as f:
            self.text = f.read()
            self.text = self.text.replace('\n','')
            return self.text
# Очистка текста        
    def clear_text(self, text):
        self.text = self.text.lower()
        self.text = self.text.replace('"', '')
        self.text = self.text.replace(',', '')
        self.text = self.text.replace(':', '')
        self.text = self.text.replace('-"', '')
        return self.text

    def create_df(self):
        self.df = pd.DataFrame(columns=['sentence', 'options', 'answer', 'task', 'result'])
        # self.text = text
        # tokens_sens = nltk.tokenize.sent_tokenize(self.text, language='english')
        # df_sentences = pd.DataFrame({'sentence': tokens_sens})
        # df_sentences["sentence"]= df_sentences.apply(lambda x: x['sentence'].replace('.', ''), axis=1)
        return self.df

# Токенизация по предложениям и создание ДФ
    def tokenization(self, text):
        self.text=text
        self.tokens_sens = nltk.tokenize.sent_tokenize(self.text, language='english')
        self.df_sentences = pd.DataFrame({'sentence': self.tokens_sens})
        self.df_sentences["sentence"]= self.df_sentences.apply(lambda x: x['sentence'].replace('.', ''), axis=1)    
        return self.df_sentences

# Обработка упражнения и вывод итогового ДФ
    def select_exercise(self, df_sentences, options, exercise_type):

        for sentence in df_sentences.sentence:
            for token in nlp(str(sentence)):
                if token.pos_=='VERB' and exercise_type == 'Выберите правильную форму глагола':
                    self.answer = [token.text for token in nlp(str(sentence)) if token.pos_=='VERB']
                    options.append(list(set([token._.inflect('VB'), token._.inflect('VBN'), token._.inflect('VBP'), token._.inflect('VBZ'), token._.inflect('VBG'), token._.inflect('VBD')])))
                    self.task = token.pos_
                    write_it_df=1
                elif token.pos_=='ADJ'and exercise_type == 'Выбор правильного прилагательного':
                    self.answer = [token.text for token in nlp(str(sentence)) if token.pos_=='ADJ']
                    options.append([token.text, token._.inflect('JJS')])
                    self.task = token.pos_
                    write_it_df=1
            
                elif exercise_type ==  'Расставьте в правильном порядке слова предложения'  and len(nlp(str(sentence))) < 9:
                    options = [token.text for token in nlp(str(sentence))]
                    options = [options] * len(options)
                    self.answer = [token.text for token in nlp(str(sentence))]
                        
                    write_it_df=1
                    self.task = 'order_words'
                else: pass                

            if exercise_type == 'Выберите правильный артикль':
                self.task = 'articles'
                self.answer=[]
                split_string = sentence.split(" ")
                #st.write(test_string)
                #st.write(len(test_string))
                if len(split_string) in range (3, 20):
                    st.write(len(split_string))
                for i in split_string:
                    for j in ['a', 'the', 'an']:
                        if i==j:
                            self.answer.append(i)
                            options.append([' a ', ' the ', ' an '])
                            break  
                self.answer = list(map(lambda x: ' '+ x + ' ', self.answer))  
                write_it_df=1     

            if len(nlp(str(sentence))) in range(3, 20) and len(self.answer) > 0 and write_it_df==1:                    
                self.df.loc[len(self.df)]=[sentence, options, self.answer, self.task, []]  

            # сбрасываем переменные    
            options=[]  
            write_it_df=0    
            self.answer=[]        
        print("Упражнение:",exercise_type)
        self.df["sentence_hidden"] = self.df["sentence"]
        for index, row in self.df.iterrows(): 
            for i in row.answer:
                print("Упражнение:",exercise_type)
                if exercise_type == 'Расставьте в правильном порядке слова предложения':
                    self.df["sentence_hidden"][index] = '_________________'
                else: self.df["sentence_hidden"][index] = self.df["sentence_hidden"][index].replace(i, ' ___ ')
        return self.df
    


In [38]:
nltk.download('punkt')

random.seed(42)

[nltk_data] Downloading package punkt to /Users/Sergey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [39]:
create_exercise = Create_exercise()   

text = st.text_area('Текст', create_exercise.get_text())

#exercise_type = st.sidebar.selectbox('Выберите тип упражнения:', ['', 'Выберите правильную форму глагола',  'Выбор правильного прилагательного', 'Выберите правильный артикль', 'Расставьте в правильном порядке слова предложения'], format_func=lambda x: 'Ничего не выбрано' if x == '' else x)
exercise_type = 'Расставьте в правильном порядке слова предложения'     

In [40]:
                                                             

# if exercise_type:
#     st.success(exercise_type)
# else:
#     st.warning('Для начала выберите в боковом меню тип упражнения')


num_of_sentenses = 5
# st.write("В заданиях будет отображаться", num_of_sentenses, 'предложений')


#get_ex.create_df(text)
text = create_exercise.clear_text(text)

In [41]:
options = []

df = create_exercise.create_df()
#print("111", exercise_type)
df_sentences = create_exercise.tokenization(text)    
df = create_exercise.select_exercise(df_sentences, options, exercise_type)

key=0

df = df[0:num_of_sentenses]

Упражнение: Расставьте в правильном порядке слова предложения
Упражнение: Расставьте в правильном порядке слова предложения
Упражнение: Расставьте в правильном порядке слова предложения
Упражнение: Расставьте в правильном порядке слова предложения
Упражнение: Расставьте в правильном порядке слова предложения
Упражнение: Расставьте в правильном порядке слова предложения
Упражнение: Расставьте в правильном порядке слова предложения
Упражнение: Расставьте в правильном порядке слова предложения
Упражнение: Расставьте в правильном порядке слова предложения
Упражнение: Расставьте в правильном порядке слова предложения
Упражнение: Расставьте в правильном порядке слова предложения
Упражнение: Расставьте в правильном порядке слова предложения
Упражнение: Расставьте в правильном порядке слова предложения
Упражнение: Расставьте в правильном порядке слова предложения
Упражнение: Расставьте в правильном порядке слова предложения
Упражнение: Расставьте в правильном порядке слова предложения
Упражнен

In [42]:
options

[]

In [43]:
exercise_type = 'Расставьте в правильном порядке слова предложения'   
exercise_type = 'Расставьте в правильном порядке слова предложения'   
exercise_type = 'Расставьте в правильном порядке слова предложения'   
#exercise_type = 'Выберите правильную форму глагола'   

In [44]:


pd.set_option('display.max_rows', 50) 

for index, row in df.iterrows():
    print(str(row['sentence_hidden']))
    print(str(row['sentence']))

    option = []
    for i in range(len(row['options'])):
        key+=1
        option = row['options'][i]
        random.shuffle(option)
        option = ['–––'] + option
        #df['result'][index] =  st.selectbox('nolabel', option, label_visibility="hidden", key =str(key) )
        df['result'][index] =  'any'

        if df['result'][index] == '–––':
            pass

        elif df['result'][index] == str(row['answer'][i]):
            print('Это правльный ответ')
        
        else:
            print('Попробуйте еще раз')
    print('----------------------')       
print(exercise_type)         

_________________
take them to your grandmother
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
----------------------
_________________
mind your manners and give her my greetings
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
----------------------
_________________
there's a hedge of hazel bushes there
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
----------------------
_________________
i'm bringing you some cake and wine
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
----------------------
_________________
i usually like it at grandmother's
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
Попробуйте еще раз
Попробуй

/var/folders/3s/0vl43zsj2n79hlj09xb7sby40000gn/T/ipykernel_4567/2359096861.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['result'][index] =  'any'


In [45]:
print(df.answer)
df

0                  [take, them, to, your, grandmother]
1    [mind, your, manners, and, give, her, my, gree...
2      [there, 's, a, hedge, of, hazel, bushes, there]
3        [i, 'm, bringing, you, some, cake, and, wine]
4          [i, usually, like, it, at, grandmother, 's]
Name: answer, dtype: object


,sentence,options,answer,task,result,sentence_hidden
0,take them to your grandmother,"[[take, them, grandmother, to, your], [take, t...","[take, them, to, your, grandmother]",order_words,any,_________________
1,mind your manners and give her my greetings,"[[greetings, her, give, manners, my, your, min...","[mind, your, manners, and, give, her, my, gree...",order_words,any,_________________
2,there's a hedge of hazel bushes there,"[[there, there, of, hedge, 's, bushes, hazel, ...","[there, 's, a, hedge, of, hazel, bushes, there]",order_words,any,_________________
3,i'm bringing you some cake and wine,"[[wine, 'm, and, i, some, bringing, you, cake]...","[i, 'm, bringing, you, some, cake, and, wine]",order_words,any,_________________
4,i usually like it at grandmother's,"[[i, 's, grandmother, it, like, at, usually], ...","[i, usually, like, it, at, grandmother, 's]",order_words,any,_________________
